# Pipeline SNPs

Se desarrolló una pipeline usando el software GATK4, bwa, Picard y SnpEff para detectar variantes (SNPs) en diferentes cultivares de aguacate, tomando de referencia el agucacate cv Hass. 

Este objetivo de tesis que consiste en detectar variantes genéticas que sirvan para clasificar los dos tipos de floración A y B en diferentes cultivares.

# STEP 0 

Preparamos los datos para trabajar:

Descargamos el genoma de aguacate Hass que se utilizara para el alineamiento, procedente de la Queensland University. Este archivo lo obtenemos de la página https://www.avocado.uma.es/easy_gdb/downloads.php en el apartado **Hass UQ** con el nombre **Avocado_Hass_genome_HiFiAsm_Gr80.fasta.gz** una vez descargado se cambia el nombre a **queensland2023.fasta**. Después usamos este archivo para crear un indice usando el software bwa. 

In [ ]:
bwa index queensland2023.fasta

Descargamos datos de dos investigaciones detallando su estructura:
Bioproyect **PRJNA564105** Talavera et al. (2019) 46 SRA utilizados 25 con floracion tipo A y 21 con tipo B.
Bioproyect **PRJNA758103** Solares et la. (2022)  26 SRA, Tipo A 11, tipo B 9.

Los datos son descargados usando fasterq-dump.


In [ ]:
fasterq-dump  SRR18503088
fasterq-dump  SRR18503073
fasterq-dump  SRR18503087
fasterq-dump  SRR18503086
fasterq-dump  SRR18503080
fasterq-dump  SRR18503072
fasterq-dump  SRR18503092
fasterq-dump  SRR18503071
fasterq-dump  SRR18503085
fasterq-dump  SRR18503070
fasterq-dump  SRR18503090
fasterq-dump  SRR18503091
fasterq-dump  SRR18503074
fasterq-dump  SRR18503084
fasterq-dump  SRR18503075
fasterq-dump  SRR18503076
fasterq-dump  SRR18503083
fasterq-dump  SRR18503082
fasterq-dump  SRR18503081
fasterq-dump  SRR18503089

# STEP   1

Mapeamos cada SRA correspondiente a un cultivar de aguacate contra el genoma de referencia de cv Hass (del cual en el paso anterior indexamos), usando bwa. Añadimos a cada cultivar una firma para identificar las variantes más adelante quedando los códigos de la siguiente manera:

In [ ]:
bwa mem -K 100000000 -t 7 -Y -R '@RG\tID:Anaheim\tLB:Anaheim\tPL:ILLUMINA\tPM:HISEQ\tSM:Anaheim' /home/synbio2/Mike_Burton/queensland2023.fasta SRR18503088_1.fastq SRR18503088_2.fastq > SRR18503088_aligned_reads.sam

Se explican los argumentos del código:
**-t** 7 le dice  BWA que se usen 7 threads para ejecutar el alineamiento.
**-Y** le dice a BWA que use recorte suave para alineaciones suplementarias.
**-K** le dice a BWA que procese bases de entrada INT en cada lote independientemente de nThreads (para reproducibilidad)
Para cada cv se añade un *readin group (RG)* con el argumento **-R**. Esta información es clave para la funcionalidad GATK descendente. GATK no funcionará sin una etiqueta de grupo de lectura (RG).

# STEP  2

Convertimos cada resultado de los alineamientos de **sam** a **bam** con el software Samtools usando un loop.

In [ ]:
for r1 in samples/*.sam 
do
samtools view -bS -F 4 "$r1" -o  "$r1".bam 
done

Unimos cada alineamiento usando Picar para trabajar con un solo archivo (por esta razón etiquetamos cada cv antes del alineamiento).

In [ ]:
picard MergeSamFiles -I SRR18503073.bam -I SRR18503080.bam -I SRR18503086.bam -I SRR18503087.bam -I SRR18503088.bam -I SRR18503072.bam -I SRR18503092.bam -I SRR18503071.bam -I SRR18503070.bam -I SRR18503085.bam -I SRR18503074.bam -I SRR18503090.bam -I SRR18503091.bam -I SRR18503075.bam -I SRR18503076.bam -I SRR18503084.bam -I SRR18503081.bam -I SRR18503082.bam -I SRR18503083.bam -I SRR18503089.bam -O output_merged_files.bam

Con este comando de samtools damos una revisada para visualizar los RG con los que etiquetamos cada cv de aguacate.

In [ ]:
samtools view -H output_merged_files.bam | grep "^@RG"

# STEP 3

Usando Picard marcados duplicados.

In [ ]:
gatk MarkDuplicatesSpark -I /home/output_merged_files.bam -M dedup_metrics.txt -O Alineamiento/sorted_dedup_reads.bam

# STEP 4

Calculamos metricas del alineamiento

In [ ]:
picard CollectAlignmentSummaryMetrics -R reference/queensland2023.fasta -I Alineamiento/sorted_dedup_reads.bam -O Alineamiento/alignment_metrics.txt

In [ ]:
picard CollectInsertSizeMetrics -I Alineamiento/sorted_dedup_reads.bam -O insert_metrics.txt -H insert_size_histogram.pdf

In [ ]:
samtools depth -a Alineamiento/sorted_dedup_reads.bam > depth_out.txt

# STEP 5

Iniciamos el Variant Calling. 

In [ ]:
gatk HaplotypeCaller -R reference/queensland2023.fasta  -I Alineamiento/sorted_dedup_reads.bam  -O raw_variants.vcf --native-pair-hmm-threads 16 -pairHMM AVX_LOGLESS_CACHING_OMP

El argumento **--native-pair-hmm-threads 16** ajusta la cantidad de threads a utilizar y **-pairHMM AVX_LOGLESS_CACHING_OMP** permite que se pueda trabajar en paralelo

NOTA:

Es posible que el programa necesite crear un indice con el archivo sorted_dedup_reads.bam y el genoma de referencia. 

In [ ]:
picard CreateSequenceDictionary -R reference/queensland2023.fasta -O queensland2023.dict

In [ ]:
samtools faidx sorted_dedup_reads.bam

# STEP 6

Extraemos SNPs.

In [ ]:
gatk SelectVariants -R reference/queensland2023.fasta -V raw_variants.vcf -select-type SNP -O raw_snps.vcf

# STEP 7

Ejecutamos un primer filtro.

In [ ]:
gatk VariantFiltration -R reference/queensland2023.fasta   -V raw_snps.vcf  -O filtered_snps.vcf -filter-name "QD_filter" -filter "QD < 2.0"   -filter-name "FS_filter" -filter "FS > 60.0" -filter-name "MQ_filter" -filter "MQ < 40.0"  -filter-name "SOR_filter" -filter "SOR > 2.0"  -filter-name "MQRankSum_filter" -filter-expression "MQRankSum<-12.5"   -filter-name "ReadPosRankSum_filter" -filter-expression "ReadPosRankSum<-8" 

Los filtros empleados son los siguientes: \
QD < 2,0: esta es la confianza de la variante (del campo QUAL) dividida por la profundidad sin filtrar de las muestras que no son de referencia homónima. Esta anotación tiene como objetivo normalizar la calidad de la variante para evitar la inflación causada cuando hay una cobertura profunda. Para fines de filtrado, es mejor utilizar QD que QUAL o DP directamente.

FS > 60,0: Esta es la probabilidad en la escala de Phred de que exista un sesgo de hebra en el sitio. Strand Bias nos dice si el alelo alternativo se observó con mayor o menor frecuencia en la cadena delantera o inversa que el alelo de referencia. Cuando hay poco o ningún sesgo de hebra en el sitio, el valor de FS será cercano a 0.

MQ < 40,0: Esta es la calidad del mapeo cuadrático medio de todas las lecturas en el sitio. En lugar de la calidad cartográfica promedio del sitio, esta anotación proporciona la raíz cuadrada del promedio de los cuadrados de las calidades cartográficas del sitio. Está destinado a incluir la desviación estándar de las cualidades cartográficas. Incluir la desviación estándar nos permite incluir la variación en el conjunto de datos. Una desviación estándar baja significa que todos los valores están cerca de la media, mientras que una desviación estándar alta significa que todos los valores están lejos de la media. Cuando las cualidades cartográficas son buenas en un sitio, el MQ será de alrededor de 60.

SOR > 4,0: esta es otra forma de estimar el sesgo de cadena utilizando una prueba similar a la prueba de odds ratio simétrica. SOR se creó porque FS tiende a penalizar las variantes que ocurren en los extremos de los exones. Las lecturas en los extremos de los exones tienden a estar cubiertas solo por lecturas en una dirección y FS otorga a esas variantes una mala puntuación. SOR tendrá en cuenta las proporciones de lecturas que cubren ambos alelos.

MQRankSum < -8.0: compara las calidades de mapeo de las lecturas que respaldan el alelo de referencia y el alelo alternativo.

ReadPosRankSum < -8.0: compara si las posiciones de los alelos de referencia y alternativos son diferentes dentro de las lecturas. Ver un alelo sólo cerca del final de las lecturas es indicativo de error, porque ahí es donde los secuenciadores tienden a cometer la mayor cantidad de errores.

# STEP 8

Excluimos variantes que no pasaron el filtro del paso anterior.

In [ ]:
gatk SelectVariants  --exclude-filtered -V filtered_snps.vcf -O bqsr_snps.vcf 

# STEP 9

GeneraMOS una tabla de recalibración para la recalibración del nivel de calidad base (BQSR).

In [ ]:
  -R reference/queensland2023.fasta \
        -I Alineamiento/sorted_dedup_reads.bam \
        --known-sites bqsr_snps.vcf \
        --known-sites bqsr_indels.vcf \
        -O recal_data.table 

# STEP 10

Aplicamos la recalibracion anterior para generar un nuevo archivo .bam para realizar un segundo variant calling.

In [ ]:
gatk ApplyBQSR -R reference/queensland2023.fasta -I All_merged/sorted_dedup_reads.bam -bqsr recal_data.table -O recal_reads.bam 

# STEP 10.1

Podemos generar una segunda recalibración y generar un reporte para comparar valores, no es obligatorio pero nosotros sí lo hicimos.

In [ ]:
gatk BaseRecalibrator \
        -R reference/queensland2023.fasta\
        -I recal_reads.bam \
        --known-sites bqsr_snps.vcf \
        --known-sites bqsr_indels.vcf \
        -O post_recal_data.table 

Generamos un plot para analizar las covarianzas.

In [ ]:
gatk AnalyzeCovariates -before recal_data.table -after post_recal_data.table  -plots recalibration_plots.pdf

# STEP 11

Ejecutamos el segundo variant calling utilizando el archivo **recal_reads.bam** generado en el *STEP 10*.

In [ ]:
gatk HaplotypeCaller -R reference/queensland2023.fasta -I recal_reads.bam -O raw_variants_recal.vcf

# STEP 12

Extraemos los nuevos SNPs.

In [ ]:
gatk SelectVariants -R reference/queensland2023.fasta -V raw_variants_recal.vcf  -select-type SNP  -O raw_snps_recal.vcf

# STEP 13

Filtramos los SNPs nuevamente.

In [ ]:
gatk VariantFiltration -R reference/queensland2023.fasta   -V raw_snps_recal.vcf  -O filtered_snps_final.vcf -filter-name "QD_filter" -filter "QD < 2.0"   -filter-name "FS_filter" -filter "FS > 60.0" -filter-name "MQ_filter" -filter "MQ < 40.0"  -filter-name "SOR_filter" -filter "SOR > 4.0"  


# STEP 14

Vamos a realizar la anotación de los SNPs, primero generamos una base de datos con los archivos de AvoBase.

In [ ]:
java -jar snpEff.jar build -gff3 -v avocadosnp2

Realizamos la anotación de las variantes con la base de datos generada.

In [ ]:
java -jar snpEff.jar avocadosnp /home/mike/Descargas/AVOCADO/obj3-SNPs/filtered_snps_final.vcf > filtered_snps_final_ann.vcf

# Hasta este punto ya tenemos los archivos vcf filtrados y anotados para ser utilizados en los análisis siguientes. 